In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K



import tensorflow as tf
import random
import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


labels.csv
sample_submission.csv
test
train



In [2]:
######### VGG19 parameters ########
dontFreezeLast = 2;

patience = 30;

loadWeights = False;
saveWeights = False;


tensorboard_dir = '../tb/catsdogs/try_batch64_size180';
os.makedirs(tensorboard_dir)
    
checkPointPath = tensorboard_dir + '/best_weights.hdf5';

####################################

gpuName = '/device:GPU:1'
tensorboardFlag = True;
workers = 10;
histogram_freq = 0;

batchSize = 64;
epochs = 100;
validation_size=0.3;

First we will read in the csv's so we can see some more information on the filenames and breeds

In [3]:
df_train = pd.read_csv('../input/labels.csv')
df_test = pd.read_csv('../input/sample_submission.csv')

print('Training images: ',df_train.shape[0])
print('Test images: ',df_test.shape[0])

# reduce dimensionality
#df_train = df_train.head(100)
#df_test = df_test.head(100)

('Training images: ', 10222)
('Test images: ', 10357)


In [4]:
df_train.head(10)

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever
5,002211c81b498ef88e1b40b9abf84e1d,bedlington_terrier
6,00290d3e1fdd27226ba27a8ce248ce85,bedlington_terrier
7,002a283a315af96eaea0e28e7163b21b,borzoi
8,003df8b8a8b05244b1d920bb6cf451f9,basenji
9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound


We can see that the breed needs to be one-hot encoded for the final submission, so we will now do this.

In [5]:
targets_series = pd.Series(df_train['breed'])
one_hot = pd.get_dummies(targets_series, sparse = True)

In [6]:
one_hot_labels = np.asarray(one_hot)

Next we will read in all of the images for test and train, using a for loop through the values of the csv files. I have also set an im_size variable which sets the size for the image to be re-sized to, 90x90 px, you should play with this number to see how it affects accuracy.

In [7]:
im_size = 120

In [8]:
x_train = []
y_train = []
x_test = []

In [9]:
i = 0 
for f, breed in tqdm(df_train.values):
    img = cv2.imread('../input/train/{}.jpg'.format(f))
    label = one_hot_labels[i]
    x_train.append(cv2.resize(img, (im_size, im_size)))
    y_train.append(label)
    i += 1

100%|██████████| 10222/10222 [00:30<00:00, 329.95it/s]


In [10]:
 for f in tqdm(df_test['id'].values):
     img = cv2.imread('../input/test/{}.jpg'.format(f))
     x_test.append(cv2.resize(img, (im_size, im_size)))

100%|██████████| 10357/10357 [00:30<00:00, 337.61it/s]


In [11]:
y_train_raw = np.array(y_train, np.uint8)
x_train_raw = np.array(x_train, np.float32) / 255.
x_test  = np.array(x_test, np.float32) / 255.

We check the shape of the outputs to make sure everyting went as expected.

In [12]:
print(x_train_raw.shape)
print(y_train_raw.shape)
print(x_test.shape)

(10222, 120, 120, 3)
(10222, 120)
(10357, 120, 120, 3)


We can see above that there are 120 different breeds. We can put this in a num_class variable below that can then be used when creating the CNN model.

In [13]:
num_class = y_train_raw.shape[1]
print('Number of classes: ', num_class)

('Number of classes: ', 120)


It is important to create a validation set so that you can gauge the performance of your model on independent data, unseen to the model in training. We do this by splitting the current training set (x_train_raw) and the corresponding labels (y_train_raw) so that we set aside 30 % of the data at random and put these in validation sets (X_valid and Y_valid).

* This split needs to be improved so that it contains images from every class, with 120 separate classes some can not be represented and so the validation score is not informative.

In [14]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=validation_size, random_state=1)

Now we build the CNN architecture. Here we are using a pre-trained model VGG19 which has already been trained to identify many different dog breeds (as well as a lot of other objects from the imagenet dataset see here for more information: http://image-net.org/about-overview). Unfortunately it doesn't seem possible to downlod the weights from within this kernel so make sure you set the weights argument to 'imagenet' and not None, as it currently is below.

We then remove the final layer and instead replace it with a single dense layer with the number of nodes corresponding to the number of breed classes we have (120).

In [15]:
# Create the base pre-trained model
# Can't download weights in the kernel
with tf.device(gpuName):
    
 
    if K.image_data_format() == 'channels_first':
        input_shape = (3, im_size, im_size)
    else:
        input_shape = (im_size, im_size, 3)

     
    
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(120))
    model.add(Activation('softmax'))

   
    ##### Mattia's model #####
    # Add a new top layer
    #x = base_model.output
    #x = Flatten()(x)
    #x = Dense(1024,activation='relu')(x)
    #x = Dense(512,activation='relu')(x)
    #predictions = Dense(num_class, activation='softmax')(x)

    # This is the model we will train
    #model = Model(inputs=base_model.input, outputs=predictions)

    # First: train only the top layers (which were randomly initialized)
    #for i in range(len(base_model.layers)-dontFreezeLast):
        #base_model.layers[i].trainable = False

    #if loadWeights:
    #    model.load_weights(checkPointPath)
    ##### Mattia's model #####
    
    model.compile(loss='categorical_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'])

    callbacks_list = [];
    callbacks_list.append(keras.callbacks.EarlyStopping(
        monitor='val_acc',
        patience=patience,
        verbose=1));
    if saveWeights:
        callbacks_list.append(keras.callbacks.ModelCheckpoint(
            checkPointPath,
            monitor='val_acc',
            verbose=1,
            save_best_only=True,
            mode='max',
            save_weights_only=True))
    if tensorboardFlag:
        callbacks_list.append(keras.callbacks.TensorBoard(
                log_dir=tensorboard_dir,
                histogram_freq=histogram_freq,
                write_graph=False,
                write_images=False));
        print('Tensorboard activated in directory: ',tensorboard_dir)
    else:
        print('Tensorboard NOT activated')


    model.summary()

('Tensorboard activated in directory: ', '../tb/catsdogs/try_batch64_size180')
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 118, 118, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 118, 118, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 59, 59, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 57, 57, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 57, 57, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 28, 28, 32)        0         
_______________________________________________________________

In [16]:
def generator(X,Y,batch_size):
    batch_features = np.ndarray(shape=(batch_size,) + X.shape[1:],
                                dtype=X.dtype);
    batch_labels = np.ndarray(shape=(batch_size,) + Y.shape[1:],
                                dtype=Y.dtype)
    N = X.shape[0];
    
    while True:
        for i in range(batch_size):
            # choose random index in features
            index= np.random.choice(N,1)
            batch_features[i] = X[index]
            batch_labels[i] = Y[index]
        yield batch_features, batch_labels

In [17]:
model.fit(X_train, Y_train,
          epochs=epochs,
          batch_size = batchSize,
          validation_data=(X_valid, Y_valid),
          verbose=1,
          callbacks=callbacks_list)

# steps_per_epoch = round(X_train.shape[0]/batchSize)
# model.fit_generator(generator(X_train,Y_train,batchSize),
#                    steps_per_epoch=steps_per_epoch,
#                    epochs=epochs,
#                    verbose=1,
#                    callbacks=callbacks_list,
#                    validation_data=(X_valid,Y_valid),
#                    workers=workers,
#                    use_multiprocessing=True)

Train on 7155 samples, validate on 3067 samples
Epoch 1/100
7155/7155 [==============================] - 10s 1ms/step - loss: 4.7887 - acc: 0.0091 - val_loss: 4.7853 - val_acc: 0.0091
Epoch 2/100
7155/7155 [==============================] - 9s 1ms/step - loss: 4.7829 - acc: 0.0110 - val_loss: 4.7752 - val_acc: 0.0157
Epoch 3/100
7155/7155 [==============================] - 9s 1ms/step - loss: 4.7480 - acc: 0.0144 - val_loss: 4.6970 - val_acc: 0.0271
Epoch 4/100
7155/7155 [==============================] - 9s 1ms/step - loss: 4.6430 - acc: 0.0247 - val_loss: 4.5608 - val_acc: 0.0258
Epoch 5/100
7155/7155 [==============================] - 9s 1ms/step - loss: 4.5322 - acc: 0.0298 - val_loss: 4.4703 - val_acc: 0.0372
Epoch 6/100
7155/7155 [==============================] - 9s 1ms/step - loss: 4.4468 - acc: 0.0363 - val_loss: 4.4271 - val_acc: 0.0372
Epoch 7/100
7155/7155 [==============================] - 9s 1ms/step - loss: 4.3679 - acc: 0.0454 - val_loss: 4.3795 - val_acc: 0.0463
Epoch 

Remember, accuracy is low here because we are not taking advantage of the pre-trained weights as they cannot be downloaded in the kernel. This means we are training the wights from scratch and I we have only run 1 epoch due to the hardware constraints in the kernel.

Next we will make our predictions.

In [18]:
 preds = model.predict(x_test, verbose=1)

10357/10357 [==============================] - 4s 428us/step


In [19]:
 sub = pd.DataFrame(preds)
 # Set column names to those generated by the one-hot encoding earlier
 col_names = one_hot.columns.values
 sub.columns = col_names
 # Insert the column id from the sample_submission at the start of the data frame
 sub.insert(0, 'id', df_test['id'])
 sub.head(10)

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,1.899682e-12,3.452019e-07,2.054581e-10,1.175180e-07,5.079506e-04,3.341352e-08,8.367716e-09,1.749750e-05,1.748507e-04,...,5.985587e-02,2.450468e-04,1.081610e-08,8.354464e-07,1.132373e-05,1.842674e-06,2.496646e-01,9.626538e-03,3.160459e-03,4.957531e-03
1,00102ee9d8eb90812350685311fe5890,3.905531e-20,3.757880e-10,1.578483e-12,4.214110e-11,6.305100e-08,5.425930e-14,4.426874e-15,1.129288e-12,5.404326e-13,...,1.019276e-01,1.862586e-05,1.212565e-14,1.067311e-09,1.663820e-06,3.044681e-10,1.617849e-01,6.900081e-03,5.322738e-03,5.161027e-05
2,0012a730dfa437f5f3613fb75efcd4ce,1.431093e-07,2.496718e-05,9.353789e-05,6.783853e-06,9.784596e-05,1.458537e-04,2.367865e-05,3.375259e-04,1.483938e-03,...,8.189092e-05,1.357693e-02,3.524707e-08,1.501845e-02,1.494005e-05,3.252132e-02,4.920302e-03,9.928199e-03,4.282502e-03,3.732287e-05
3,001510bc8570bbeee98c8d80c8a95ec1,9.830560e-06,1.430122e-07,2.776851e-04,2.569378e-03,4.152619e-10,3.701961e-12,3.652347e-02,1.618653e-04,4.853327e-04,...,6.905715e-06,1.694879e-07,2.928152e-08,1.145460e-13,2.456031e-07,8.307848e-16,3.911969e-11,4.784546e-07,4.519426e-11,2.154826e-03
4,001a5f3114548acdefa3d4da05474c2e,7.215678e-11,1.482423e-06,8.406336e-10,6.709536e-13,1.491431e-06,1.644103e-07,3.792244e-13,4.048124e-14,3.273212e-13,...,6.611486e-10,1.539033e-03,1.083425e-21,3.245802e-09,3.013204e-04,3.314551e-09,5.339412e-07,1.367623e-06,8.736789e-10,1.726707e-10
5,00225dcd3e4d2410dd53239f95c0352f,1.996567e-02,3.567657e-02,3.849713e-07,1.577741e-02,2.250122e-02,3.199835e-07,1.919563e-02,3.517269e-07,2.714097e-06,...,2.513383e-02,2.824181e-09,2.492561e-06,5.616707e-08,1.900309e-02,1.549820e-08,3.333832e-11,2.074663e-08,1.845767e-06,2.570933e-02
6,002c2a3117c2193b4d26400ce431eebd,3.539164e-04,1.280201e-02,7.644175e-04,7.405905e-02,1.026990e-02,3.852123e-05,1.674672e-03,1.038244e-03,6.686410e-03,...,9.597536e-02,1.997228e-06,1.541496e-02,3.822189e-04,2.421802e-02,2.573287e-04,1.060835e-06,2.380389e-03,3.900470e-04,5.996899e-02
7,002c58d413a521ae8d1a5daeb35fc803,1.504720e-21,7.210201e-07,1.137550e-24,9.011653e-16,7.574295e-10,3.767510e-15,1.015742e-22,4.653347e-11,2.903910e-14,...,2.740055e-09,2.414340e-12,6.723883e-28,1.953625e-06,8.612848e-07,6.875182e-04,6.713722e-03,7.042079e-08,4.734863e-08,5.360243e-10
8,002f80396f1e3db687c5932d7978b196,2.767626e-03,9.552929e-03,1.438125e-04,6.823797e-04,2.969588e-03,1.748249e-02,2.023792e-05,2.383215e-03,4.799242e-02,...,9.647001e-02,9.175779e-04,3.386614e-03,1.067411e-02,1.967616e-03,4.807285e-04,1.061835e-02,1.857554e-03,1.374490e-04,1.778218e-03
9,0036c6bcec6031be9e62a257b1c3c442,3.947569e-09,1.312877e-02,4.409037e-04,1.532943e-04,1.155733e-03,1.452565e-05,4.979119e-05,6.186082e-02,1.237942e-02,...,9.567019e-03,2.073247e-05,4.465287e-04,2.327493e-04,6.182371e-05,4.963567e-02,5.814286e-06,3.545842e-02,2.596299e-03,3.247610e-02
